# Ranking for property

In [187]:
import sys
sys.path.append("../repository/")
import database as db
import polars as pl
from math import floor

In [188]:
conn_wgi = db.DatabaseConnection("world-governance-index").connection
conn_ti = db.DatabaseConnection("transformation-index").connection
conn_ef = db.DatabaseConnection("economic-freedom").connection
conn_db = db.DatabaseConnection("doing-bussiness").connection




In [189]:
db_rank_registering_property = pl.read_database(
        query="""
        SELECT country_id, rank_registering_property
        FROM "freedom"
        WHERE YEAR = 2020
        ORDER BY rank_registering_property
        """,
        connection=conn_db,
    )

ef_property_rights = pl.read_database(
        query="""
        SELECT country_id, real_property, property_rights
        FROM "freedom"
        WHERE YEAR = 2020
        ORDER BY country_id
        """,
        connection=conn_ef,
    )

ti_property_rights = pl.read_database(
        query="""
        SELECT country_id, property_rights, private_property
        FROM "freedom"
        WHERE YEAR = 2020
        ORDER BY country_id
        """,
        connection=conn_ti,
    )

ef_weights = [1.5, 1.0]
ti_weights = [0.5, 0.5]
db_weight = 3.5



In [190]:
ef_property_rights = ef_property_rights.with_columns(
    pl.fold(
        acc=pl.lit(0),
        function=lambda c1, c2: c1 + c2,
        exprs=[
            pl.col(col) * pl.lit(wgt)
            for col, wgt in zip(ef_property_rights.columns[1:], ef_weights)
        ],
    ).alias("score")
).sort(by="score", descending=True)

ef_property_rights = ef_property_rights.with_columns(
    ef_rank=ef_property_rights["score"].rank("ordinal", descending=True)
).select(pl.col(["country_id", "ef_rank"]))


ti_property_rights = ti_property_rights.with_columns(
    pl.sum_horizontal(pl.exclude("country_id")).rank("ordinal").alias("ti_rank")
).select(pl.col(["country_id", "ti_rank"]))


db_rank_registering_property = db_rank_registering_property.rename(
    {"rank_registering_property": "db_rank"}
)


df = db_rank_registering_property.join(
    ef_property_rights, how="left", on="country_id"
).join(ti_property_rights, how="left", on="country_id")
df = df.sort(by=["ef_rank", "db_rank", "ti_rank"], descending=False, nulls_last=True)

In [191]:
try:
    ranking = df.select(pl.arg_where(pl.col("country_id") == 85)).item()
    status = "best" if ranking <= floor(len(df) / 2) else "worst"
    percentage = (
        ranking / len(df) * 100 if status is "best" else 100 - ranking / len(df) * 100
    )
    comparison = (
        "Among the **{:.1f}% {}** countries in safeguarding Property Rights.".format(
            percentage, status
        )
    )
except ValueError:
    ranking, comparison = 0, ""
comparison

<>:5: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:5: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_3100/3714414455.py:5: SyntaxWarning: "is" with a literal. Did you mean "=="?
  ranking / len(df) * 100 if status is "best" else 100 - ranking / len(df) * 100


''